In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import tensorflow as tf
from tensorflow.python.estimator import estimator
from tensorflow.python.estimator import model_fn

import numpy as np

/Users/teggsung/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [5]:
'''Overall structure:

    input: two different dataset
        Call dataset operations
        densenet: L=200, k=12, dropout=0; ep=300, batch_size=64
        wideresnet: depth=28, width=10, dropout=0; ep=200, batch_size=128
        
    modules: input perturbation
             temperature scaling
             
    op: SGD with Nesterov momentum (0.9)
    lr: start=0.1, decay_rate=0.1, decay_time=[50,75]
    
    in-distribution datasets: CIFAR-10, CIFAR-100
    out-of-distribution datasets: TinyImageNet, LSUN, iSUN, Gaussian noise, Uniform noise
'''

'\nOverall structure:\n\n    input: two different dataset\n        Call dataset operations\n        densenet: L=200, k=12, dropout=0; ep=300, batch_size=64\n        wideresnet: depth=28, width=10, dropout=0; ep=200, batch_size=128\n        \n    modules: input perturbation\n             temperature scaling\n             \n    op: SGD with Nesterov momentum (0.9)\n    lr: start=0.1, decay_rate=0.1, decay_time=[50,75]\n    \n    in-distribution datasets: CIFAR-10, CIFAR-100\n    out-of-distribution datasets: TinyImageNet, LSUN, iSUN, Gaussian noise, Uniform noise\n'

In [17]:
## Temperature scaling
def temp_scaling(x, model, temperature):
    scaled = tf.nn.softmax_cross_entropy_with_logits_v2(x, model(x)/temperature)
    return scaled

In [19]:
## Image preprocessing
def preprocess(x, model, eps, temperature):
    perturbed = temp_scaling(x, model, temperature)
    prep_x = x - eps * np.sign(-perturbed)
    return prep_x

In [38]:
## Compare score
def check_in_out(x, y, model, thresh, temperature=1000, eps=0.0012):
    score = tf.nn.softmax_cross_entropy_with_logits_v2(preprocess(x, model, eps, temperature), y)
    if score >= thresh:
        print('in-distribution')
    else:
        print('out-of-distribution')

# Model code

In [3]:
## Reference: https://github.com/tensorflow/models/blob/master/official/mnist/mnist.py
## TODO: Need to create method for loadinng pre-trained model?

def create_model(data_format):
    """Model to recognize digits in the MNIST dataset.
    Network structure is equivalent to:
    https://github.com/tensorflow/tensorflow/blob/r1.5/tensorflow/examples/tutorials/mnist/mnist_deep.py
    and
    https://github.com/tensorflow/models/blob/master/tutorials/image/mnist/convolutional.py
    But uses the tf.keras API.
    Args:
    data_format: Either 'channels_first' or 'channels_last'. 'channels_first' is
      typically faster on GPUs while 'channels_last' is typically faster on
      CPUs. See
      https://www.tensorflow.org/performance/performance_guide#data_formats
    Returns:
    A tf.keras.Model.
    """
    if data_format == 'channels_first':
        input_shape = [1, 28, 28]
    else:
        assert data_format == 'channels_last'
        input_shape = [28, 28, 1]
    
    
    ## TODO: Transfer TF to keras
    '''From Densenet TF official code'''
    def conv(image, filters, strides=1, kernel_size=3):
    """Convolution with default options from the densenet paper."""
    # Use initialization from https://arxiv.org/pdf/1502.01852.pdf

    return tf.layers.conv2d(
        inputs=image,
        filters=filters,
        kernel_size=kernel_size,
        strides=strides,
        activation=tf.identity,
        use_bias=False,
        padding="same",
        kernel_initializer=tf.variance_scaling_initializer(),
    )

    def dense_block(image, filters, is_training):
    """Standard BN+Relu+conv block for DenseNet."""
    image = tf.layers.batch_normalization(
        inputs=image,
        axis=-1,
        training=is_training,
        fused=True,
        center=True,
        scale=True,
        momentum=_BATCH_NORM_DECAY,
        epsilon=_BATCH_NORM_EPSILON,
    )

    if FLAGS.use_bottleneck:
        # Add bottleneck layer to optimize computation and reduce HBM space
        image = tf.nn.relu(image)
        image = conv(image, 4 * filters, strides=1, kernel_size=1)
        image = tf.layers.batch_normalization(
            inputs=image,
            axis=-1,
            training=is_training,
            fused=True,
            center=True,
            scale=True,
            momentum=_BATCH_NORM_DECAY,
            epsilon=_BATCH_NORM_EPSILON,
        )

    image = tf.nn.relu(image)
    return conv(image, filters)


    def transition_layer(image, filters, is_training):
    """Construct the transition layer with specified growth rate."""

    image = tf.layers.batch_normalization(
        inputs=image,
        axis=-1,
        training=is_training,
        fused=True,
        center=True,
        scale=True,
        momentum=_BATCH_NORM_DECAY,
        epsilon=_BATCH_NORM_EPSILON,
    )
    image = tf.nn.relu(image)
    conv_img = conv(image, filters=filters, kernel_size=1)
    return tf.layers.average_pooling2d(
        conv_img, pool_size=2, strides=2, padding="same")


    def _int_shape(layer):
        return layer.get_shape().as_list()


    # Definition of the CIFAR-10 network
    def densenet_cifar_model(image,
                             k,
                             layers,
                             is_training=True,
                             num_blocks=3,
                             num_classes=10):
        """Construct a DenseNet with the specified growth size and layers."""
        layers_per_block = int((layers - 4) / num_blocks)

        v = conv(image, filters=2*k, strides=(1, 1), kernel_size=(3, 3))
        for i in range(num_blocks):
            with tf.variable_scope("block-%d" % i):
                for j in range(layers_per_block):
                    with tf.variable_scope("conv-%d-%d" % (i, j)):
                        dv = dense_block(v, k, is_training)
                        v = tf.concat([v, dv], axis=3)
            if i != num_blocks - 1:
                with tf.variable_scope("transition-%d" % i):
                    v = transition_layer(v, _int_shape(v)[3], is_training)

        global_pool = tf.reduce_sum(v, axis=(2, 3), name="global_pool")
        logits = tf.layers.dense(
            global_pool,
            units=num_classes,
            activation=tf.identity,
            kernel_initializer=tf.random_normal_initializer(stddev=2.0 / (
                _int_shape(global_pool)[1] * 10)),
            kernel_regularizer=tf.contrib.layers.l2_regularizer(1e-4),
            bias_regularizer=tf.contrib.layers.l2_regularizer(1e-4),
        )
        return logits

'''Previous Keras MNIST code'''
    l = tf.keras.layers
    max_pool = l.MaxPooling2D(
        (2, 2), (2, 2), padding='same', data_format=data_format)
    # The model consists of a sequential chain of layers, so tf.keras.Sequential
    # (a subclass of tf.keras.Model) makes for a compact description.
    return tf.keras.Sequential(
        [
            l.Reshape(
                target_shape=input_shape,
                input_shape=(28 * 28,)),
            l.Conv2D(
                32,
                5,
                padding='same',
                data_format=data_format,
                activation=tf.nn.relu),
            max_pool,
            l.Conv2D(
                64,
                5,
                padding='same',
                data_format=data_format,
                activation=tf.nn.relu),
            max_pool,
            l.Flatten(),
            l.Dense(1024, activation=tf.nn.relu),
            l.Dropout(0.4),
            l.Dense(10)
        ])

def model_fn(features, labels, mode, params):
    """The model_fn argument for creating an Estimator."""
    model = create_model(params['data_format'])
    image = features
    if isinstance(image, dict):
        image = features['image']

    if mode == tf.estimator.ModeKeys.PREDICT:
        logits = model(image, training=False)
        predictions = {
            'classes': tf.argmax(logits, axis=1),
            'probabilities': tf.nn.softmax(logits),
        }
        return tf.estimator.EstimatorSpec(
            mode=tf.estimator.ModeKeys.PREDICT,
            predictions=predictions,
            export_outputs={
                'classify': tf.estimator.export.PredictOutput(predictions)
            })
    
    if mode == tf.estimator.ModeKeys.TRAIN:
        optimizer = tf.train.AdamOptimizer(learning_rate=params['learning_rate'])

        logits = model(image, training=True)
        loss = tf.losses.sparse_softmax_cross_entropy(labels=labels, logits=logits)
        accuracy = tf.metrics.accuracy(
            labels=labels, predictions=tf.argmax(logits, axis=1))

        # Name tensors to be logged with LoggingTensorHook.
        tf.identity(LEARNING_RATE, 'learning_rate')
        tf.identity(loss, 'cross_entropy')
        tf.identity(accuracy[1], name='train_accuracy')

        # Save accuracy scalar to Tensorboard output.
        tf.summary.scalar('train_accuracy', accuracy[1])

        return tf.estimator.EstimatorSpec(
            mode=tf.estimator.ModeKeys.TRAIN,
            loss=loss,
            train_op=optimizer.minimize(loss, tf.train.get_or_create_global_step()))
    
    if mode == tf.estimator.ModeKeys.EVAL:
        logits = model(image, training=False)
        loss = tf.losses.sparse_softmax_cross_entropy(labels=labels, logits=logits)
        return tf.estimator.EstimatorSpec(
            mode=tf.estimator.ModeKeys.EVAL,
            loss=loss,
            eval_metric_ops={
                'accuracy': tf.metrics.accuracy(
                                labels=labels, predictions=tf.argmax(logits, axis=1)),
            })

# def cnn_model_fn(features, labels, mode):
#     input_layer = features['images']
    
#     conv1 = tf.layers.conv2d(
#         inputs=input_layer,
#         filters=32,
#         kernel_size=[5,5],
#         padding='same',
#         activation=None)
#     batch_norm1 = tf.layers.batch_normalization(conv1)
#     relu1 = tf.nn.relu(batch_norm1)
#     pool1 = tf.layers.max_pooling2d(inputs=relu1, pool_size=[2,2], strides=2)
    
#     conv2 = tf.layers.conv2d(
#         inputs=pool1,
#         filters=64,
#         kernel_size=[5,5],
#         padding='same',
#         activation=None)
#     batch_norm2 = tf.layers.batch_normalization(conv2)
#     relu2 = tf.nn.relu(batch_norm2)
#     pool2 = tf.layers.max_pooling2d(inputs=relu2, pool_size=[2,2], strides=2)
    
#     pool2_flat = tf.layers.flatten(pool2)
    
#     dense = tf.layers.dense(inputs=pool2_flat, units=1024, activation=tf.nn.relu)
    
#     dropout = tf.layers.dropout(inputs=dense, rate=0.4, 
#                                 training={mode == tf.estimator.ModeKeys.TRAIN})
    
#     logits = tf.layers.dense(inputs=dropout, units=params['num_classes'])
    
#     predictions = {'classes': tf.argmax(input=logits, axis=1),
#                    'probabilities': tf.nn.softmax(logits, name='softmax_tensor')}
    
#     if mode == tf.estimator.ModeKeys.PREDICT:
#         spec = tf.estimator.EstimatorSpec(
#             mode=mode, predictions=predictions, 
#             export_outputs={'predict': tf.estimator.export.PredictOutput(predictions)})
#         return spec
        
#     loss = tf.losses.sparse_softmax_cross_entropy(labels=labels, logits=logits)
    
#     if mode == tf.estimator.ModeKeys.TRAIN:
#         spec = tf.estimator.EstimatorSpec(
#             optimizer = tf.train.AdamOptimizer(),
#             train_op = optimizer.minimize(loss=loss, global_step=tf.train.get_global_step()) )
#         return spec
        
#     eval_metric_ops = {
#         'accuracy': tf.metrics.accuracy(labels=labels, predictions=predictions['classes']),
#         'aupr': tf.metrics.average_precision_at_k(labels, predictions, k=k)
#     }
#     spec = tf.estimator.EstimatorSpec(mode=mode, loss=loss, eval_metric_ops=eval_metric_ops)
#     return spec

In [4]:
def odin_classifier(dataset, model, mode, params):
#     labels = tf.cast(labels, tf.int32)
    
    ## Check available mode
    if mode not in [model_fn.ModeKeys.TRAIN, model_fn.ModeKeys.EVAL, model_fn.ModeKeys.PREDICT]:
        raise ValueError('Mode not recognized: {}'.format(mode))
        
#    assert isinstance() or isinstance()
    
    check_in(x=dataset1,
             y=dataset2,
             model=model,
             thresh=0.5)
    
    # Define training operations
    op =
    loss = 
    
    # Define accuracy, metrics
    metrics = 
    accuracy = 
    
    
    # Eval
    if mode == tf.estimator.ModeKeys.EVAL:
        spec = tf.estimator.EstimatorSpec(
            mode, loss=loss, eval_metric_ops=metrics)
        merged = tf.summary.merge_all()
        test_writer = tf.summary.FileWriter('test')
        return spec

    # Train: Alternative learning
    if mode == tf.estimator.ModeKeys.TRAIN:
        spec = tf.estimator.EstimatorSpec(
            mode, loss=loss, train_op=op)
        tf.summary.scalar('accuracy', accuracy[1])
        merged = tf.summary.merge_all()
        train_writer = tf.summary.FileWriter('train')
        return spec

    raise ValueError("Invalid estimator mode: reached the end of the function")

SyntaxError: invalid syntax (<ipython-input-4-909a226d874c>, line 16)

# Example code

In [2]:
## Metric
tf.metrics.auc(labels,  predictionns)
tf.metrics.accuracy(labels, predictions)
tf.metrics.precision(labels, predictions)
tf.metrics.recall(labels, predictions)
tf.metrics.average_precision_at_k(labels, predictions, k)

NameError: name 'labels' is not defined

In [2]:
params = {
    'steps': 10000,
    'num_classes': 10,
    'model_dir': './ckpt/',
    'saved_dir': '/pb/',
    'learning_rate': 1e-4,
}

In [8]:
''' dataset1 = ...
    dataset2 = ...
    odin_clasifier(dataset1, dataset2)
'''

' dataset1 = ...\n    dataset2 = ...\n    odin_clasifier(dataset1, dataset2)\n'

In [21]:
training_config = tf.estimator.RunConfig(
    model_dir=params['model_dir'],
    save_summary_steps=20,
    save_checkpoints_steps=20)
classifier = tf.estimator.Estimator(
    config=training_config,
    model_fn=cnn_model_fn,
    model_dir=params['model_dir'])

INFO:tensorflow:Using config: {'_model_dir': './ckpt/', '_tf_random_seed': None, '_save_summary_steps': 20, '_save_checkpoints_steps': 20, '_save_checkpoints_secs': None, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0xb20e46ef0>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [22]:
tensors_to_log = {'probabilities': 'softmax_tensors'}
logging_hook = tf.train.LoggingTensorHook(tensors=tensors_to_log, every_n_iter=50)

In [23]:
train_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={'images': train_data},
    y=train_labels,
    batch_size=params['steps'],
    num_epochs=None,
    shuffle=True)
classifier.train(input_fn=train_input_fn, 
                 steps=params['steps'], 
                 hooks=[logging_hook])

eval_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={'image': eval_data},
    y=eval_labels,
    num_epochs=1,
    shuffle=False)
eval_results = classifier.evaluate(input_fn=eval_input_fn)
print(eval_results)

NameError: name 'train_data' is not defined